### Estruturas Criptográficas - 2022-2023
### Grupo 7
#### TP1. Problema 2

Pretende-se criar uma cifra com autenticação de meta-dados a partir de um gerador pseudo-aleatório do tipo XOF. Este gerador, que tem como "seed" uma chave gerada por um KDF a partir de uma password fornecida, produzirá uma sequência de palavras ("outputs") de 64 bits. A mensagem será cifrada por blocos, i.e. será numa primeira fase dividida, e cada bloco será cifrado com o respetivo "output" gerado pelo gerador. Uma vez cifrada a mensagem, esta juntamente com os meta-dados serão autenticados utilizando a própria "seed" do gerador (a chave obtida pela KDF).

In [95]:
import os
from pickle import dumps
from cryptography.hazmat.primitives import hashes, hmac, padding
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

In [96]:
def generate_cipher_key(password):
    kdf = PBKDF2HMAC(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = os.urandom(16),
        iterations = 480000,
    )
    return kdf.derive(password)

def generator(seed, n):
    digest = hashes.Hash(hashes.SHAKE256(2**n * 8))
    digest.update(seed)
    return digest.finalize()

In [99]:
def encrypt(plaintext, outputs, auth_key):
    blocks = []
    for i in range(0, len(plaintext), 8):
        block = plaintext[i:i+8]
        # padding, com zeros
        if len(block) < 8:
            block += b'\x00' * (8 - len(block))
        blocks.append(block)
        
    ciphertext = b''
    for block, output in zip(blocks, outputs):
        for b, o in zip(block, output):
            ciphertext = b''

    h = hmac.HMAC(auth_key, hashes.SHAKE256(), backend = default_backend())
    h.update(ciphertext)

    return (ciphertext, h.finalize())
        

def decrypt(ciphertext, outputs, signature, auth_key):
    h = hmac.HMAC(auth_key, hashes.SHAKE256(), backend = default_backend())
    h.update(ciphertext)
    try:
        h.verify(signature)
    except InvalidSignature:
    else:
        # decrypt ciphertext

    return 0

In [103]:
def run():

    message = input('Enter the message: ')
    print('Message: ' + message)
    n = int(input('Enter the n: '))
    password = input('Enter the password: ').encode('utf-8')

    cipher_key = generate_cipher_key(password)
    words = generator(cipher_key, n)

    # outputs = [int.from_bytes(words[i:i+8], byteorder='big') for i in range(0, len(words), 8)]
    outputs = [words[i:i+8] for i in range(0, len(words), 8)]

    auth_key = b'accorded authentication key'
    x = encrypt(bytes(message, 'utf-8'), outputs, auth_key)

    ciphertext = x[0]
    signature = x[1]

    final_message = decrypt(ciphertext, outputs, signature, auth_key)

run()

Message: mafalda
b'\x073\xeb|d\x1f\xaf\xa3\xac\xf0\xb4\x14W|quW\x088\x04w\xab\t/\x84 )\x00\xa8l\xc2\x99{\x1b[\xff\x12\x01(\xef\xbfI\xb7\xc0(\xcc\x89\xda\xfeU\xe8\xec|u\xdc9w\x18\x94\x84\xfc\x1e\xc5F\xbe\xfe,\xb8\x8e\xee\x9c\xddGf\x8d\xcc>\x0b\xa0\xc5\xcb;\xb5\xc3\x8b\x93\xc35w\xd6e\xe1\x89\xa6\x92\xbc\x88x\xef\x8a\xe8I\xab&\xcc\xfe\xd0\xa1\xd7\x84w\xfcX\x90y\x89`\x0b\x02\x8e\xfb\x82\xb7^#X\xe5\r'
-
[519017301566009251, 12461658168394281333, 6271324072952138031, 9520654695063667353, 8870785042113177839, 13783750170490210778, 18326810361052126265, 8581772388840949062, 13762486682474421469, 5144955532671164613, 14644498464790266677, 8635201354880946876, 9833873166245145382, 14771473221345900540, 6381734302912807566, 18123249465440134413]
-
[b'\x073\xeb|d\x1f\xaf\xa3', b'\xac\xf0\xb4\x14W|qu', b'W\x088\x04w\xab\t/', b'\x84 )\x00\xa8l\xc2\x99', b'{\x1b[\xff\x12\x01(\xef', b'\xbfI\xb7\xc0(\xcc\x89\xda', b'\xfeU\xe8\xec|u\xdc9', b'w\x18\x94\x84\xfc\x1e\xc5F', b'\xbe\xfe,\xb8\x8e\xee\x9c\xdd',